# ItaData2024

## Pneumonia

propositional analysis

In [1]:
using Pkg
Pkg.activate(".")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22
using Test
using Plots

  Activating project at `~/Documents/Aclai/audio-rules2024`


### Settings

In [2]:
experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

features = :catch9
# features = :minmax
# features = :custom

# loadset = false
loadset = true

scale = :semitones
# scale = :mel_htk

sr = 8000

featset = ()
# featset = (:mfcc,)
# featset = (:f0,)
# featset = (:mfcc, :f0)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in featset ? false : true,
    audio_norm = true,
)

memguard = false;
# memguard = true;
n_elems = 15;

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]

@assert experiment in avail_exp "Unknown type of experiment: $experiment."

findhealthy = y -> findall(x -> x == "Healthy", y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
findsick = y -> findall(x -> x == String(experiment), y)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"
memguard && begin filename *= string("_memguard") end

destpath = "results/propositional"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment) * "_" * String(scale) * ".jld2"

color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36);
r_select = r"\e\[\d+m(.*?)\e\[0m";

### Prepare dataset for analysis

In [3]:
d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

memguard && begin
    cat2 = round(Int, length(y)/2)
    indices = [1:n_elems; cat2:cat2+n_elems-1]
    x = x[indices, :]
    y = y[indices]
end

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

catch9_f = ["max", "min", "mean", "med", "std", "bsm", "bsd", "qnt", "3ac"]
variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)
    
catch9 = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    std,
    Catch22.SB_BinaryStats_mean_longstretch1,
    Catch22.SB_BinaryStats_diff_longstretch0,
    Catch22.SB_MotifThree_quantile_hh,
    Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]

if !loadset
    @info("Build dataset...")

    X = DataFrame([name => Float64[] for name in [match(r_select, v)[1] for v in variable_names]])
    audiofeats = [afe(row[:audio]; featset=featset, audioparams...) for row in eachrow(x)]
    push!(X, vcat([vcat([map(func, eachcol(row)) for func in catch9]...) for row in audiofeats])...)

    yc = CategoricalArray(y);

    train_ratio = 0.8

    train, test = partition(eachindex(yc), train_ratio, shuffle=true)
    X_train, y_train = X[train, :], yc[train]
    X_test, y_test = X[test, :], yc[test]
    save(dsfile, Dict("X_test" => X_test, "y_test" => y_test))

    println("Training set size: ", size(X_train), " - ", length(y_train))
    println("Test set size: ", size(X_test), " - ", length(y_test))
end

### Train a model

In [4]:
if !loadset
    learned_dt_tree = begin
        Tree = MLJ.@load DecisionTreeClassifier pkg=DecisionTree
        model = Tree(max_depth=-1, )
        mach = machine(model, X_train, y_train)
        fit!(mach)
        fitted_params(mach).tree
    end
end

### Model inspection & rule study

In [5]:
if !loadset
    sole_dt = solemodel(learned_dt_tree)
    # Make test instances flow into the model, so that test metrics can, then, be computed.
    apply!(sole_dt, X_test, y_test);
    # Save solemodel to disk
    save(jld2file, Dict("sole_dt" => sole_dt))
else
    @info("Load dataset...")
    d = jldopen(dsfile)
    X_test = d["X_test"]
    y_test = d["y_test"]
    close(d)
    d = jldopen(jld2file)
    sole_dt = d["sole_dt"]
    close(d)
end
# Print Sole model
printmodel(sole_dt; show_metrics = true, variable_names_map = variable_names);

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X14sZmlsZQ==.jl:8


▣ max(cntrd) < 1175.142326739371
├✔ min(mel8=1194Hz) < -4.254264886244089
│├✔ std(mel13=2832Hz) < 0.12871913235095664
││├✔ std(mel9=1419Hz) < 0.27929614257337987
│││├✔ Pneumonia : (ninstances = 28, ncovered = 28, confidence = 0.96, lift = 1.0)
│││└✘ med(cntrd) < 692.5990090205792
│││ ├✔ med(mel8=1194Hz) < -5.408287525000148
│││ │├✔ Healthy : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
│││ │└✘ Pneumonia : (ninstances = 6, ncovered = 6, confidence = 1.0, lift = 1.0)
│││ └✘ Healthy : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
││└✘ min(mel11=2005Hz) < -7.039873889080836
││ ├✔ Healthy : (ninstances = 5, ncovered = 5, confidence = 0.8, lift = 1.0)
││ └✘ min(kurts) < 5.440264768459491
││  ├✔ med(flatn) < 0.013978272288206726
││  │├✔ Healthy : (ninstances = 2, ncovered = 2, confidence = 1.0, lift = 1.0)
││  │└✘ std(mel6=845Hz) < 0.5040328419321546
││  │ ├✔ mean(mel11=2005Hz) < -5.921476625324498
││  │ │├✔ Pneumonia : (ninstances = 0, ncovered = 0, confidence 

### Extract rules that are at least as good as a random baseline model

In [6]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map = variable_names);

▣ (max(cntrd) < 1175.142326739371) ∧ (min(mel8=1194Hz) < -4.254264886244089) ∧ (std(mel13=2832Hz) < 0.12871913235095664) ∧ (std(mel9=1419Hz) < 0.27929614257337987)  ↣  Pneumonia : (ninstances = 101, ncovered = 28, coverage = 0.28, confidence = 0.96, lift = 1.91, natoms = 4)
▣ (max(cntrd) < 1175.142326739371) ∧ (min(mel8=1194Hz) < -4.254264886244089) ∧ (std(mel13=2832Hz) < 0.12871913235095664) ∧ (¬(std(mel9=1419Hz) < 0.27929614257337987)) ∧ (med(cntrd) < 692.5990090205792) ∧ (¬(med(mel8=1194Hz) < -5.408287525000148))  ↣  Pneumonia : (ninstances = 101, ncovered = 6, coverage = 0.06, confidence = 1.0, lift = 1.98, natoms = 6)
▣ (max(cntrd) < 1175.142326739371) ∧ (min(mel8=1194Hz) < -4.254264886244089) ∧ (std(mel13=2832Hz) < 0.12871913235095664) ∧ (¬(std(mel9=1419Hz) < 0.27929614257337987)) ∧ (¬(med(cntrd) < 692.5990090205792))  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 2.02, natoms = 5)
▣ (max(cntrd) < 1175.142326739371) ∧ (min(mel8=1194Hz) < 

### Simplify rules while extracting and prettify result

In [7]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map = variable_names);

▣ (max(cntrd) < 1175.14) ∧ (min(mel8=1194Hz) < -4.25) ∧ (std(mel13=2832Hz) < 0.13) ∧ (std(mel9=1419Hz) < 0.28)  ↣  Pneumonia : (ninstances = 101, ncovered = 28, coverage = 0.28, confidence = 0.96, lift = 1.91, natoms = 4)
▣ (max(cntrd) < 1175.14) ∧ (min(mel8=1194Hz) < -4.25) ∧ (std(mel13=2832Hz) < 0.13) ∧ (std(mel9=1419Hz) ≥ 0.28) ∧ (med(cntrd) < 692.6) ∧ (med(mel8=1194Hz) ≥ -5.41)  ↣  Pneumonia : (ninstances = 101, ncovered = 6, coverage = 0.06, confidence = 1.0, lift = 1.98, natoms = 6)
▣ (max(cntrd) < 1175.14) ∧ (min(mel8=1194Hz) < -4.25) ∧ (std(mel13=2832Hz) < 0.13) ∧ (std(mel9=1419Hz) ≥ 0.28) ∧ (med(cntrd) ≥ 692.6)  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 2.02, natoms = 5)
▣ (max(cntrd) < 1175.14) ∧ (min(mel8=1194Hz) < -4.25) ∧ (std(mel13=2832Hz) ≥ 0.13) ∧ (min(mel11=2005Hz) < -7.04)  ↣  Healthy : (ninstances = 101, ncovered = 5, coverage = 0.05, confidence = 0.8, lift = 1.62, natoms = 4)
▣ (max(cntrd) < 1175.14) ∧ (min(mel8=1194Hz) 

### Directly access rule metrics

In [8]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

14-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 101, ncovered = 28, coverage = 0.27722772277227725, confidence = 0.9642857142857143, lift = 1.9096638655462186, natoms = 4)
 (ninstances = 101, ncovered = 6, coverage = 0.0594059405940594, confidence = 1.0, lift = 1.9803921568627452, natoms = 6)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.02, natoms = 5)
 (ninstances = 101, ncovered = 5, coverage = 0.04950495049504951, confidence = 0.8, lift = 1.616, natoms = 4)
 (ninstances = 101, ncovered = 2, coverage = 0.019801980198019802, confidence = 1.0, lift = 2.02, natoms = 6)
 (ninstances = 101, ncovered = 2, coverage = 0.019801980198019802, confidence = 1.0, lift = 2.02, natoms = 8)
 (ninstances = 101, ncovered = 2, coverage = 0.019801980198019802, confidence = 1.0, lift = 1.9803921568627452, natoms = 9)
 (ninstances = 101, ncovered = 6, cov

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [9]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map = variable_names);

▣ (max(cntrd) < 1175.142326739371) ∧ (min(mel8=1194Hz) ≥ -4.254264886244089) ∧ (min(sprd) ≥ 318.63517531591276)  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.9803921568627452, natoms = 3, height = 2)
▣ (max(cntrd) < 1175.142326739371) ∧ (min(mel8=1194Hz) < -4.254264886244089) ∧ (std(mel13=2832Hz) < 0.12871913235095664) ∧ (std(mel9=1419Hz) ≥ 0.27929614257337987) ∧ (med(cntrd) ≥ 692.5990090205792)  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.02, natoms = 5, height = 4)
▣ (max(cntrd) ≥ 1175.142326739371) ∧ (std(flatn) < 0.05702248013266628) ∧ (med(decrs) ≥ -0.23021399168102558) ∧ (std(cntrd) < 152.23591219501) ∧ (bsm(entrp) ≥ 6.0)  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.02, natoms = 5, height = 4)
▣ (max(cntrd) ≥ 1175.142326739371) ∧ (std(flatn) < 0.05702248013266628) ∧ (med(decrs) < -0.2302139916810

### Pretty table of rules and their metrics

In [10]:
metricstable(interesting_rules; variable_names_map = variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────┬──────────┬────────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                                                                                                                                                                                                  Antecedent │ Consequent │ ninstances │ ncovered │   coverage │ confidence │    lift │ natoms │ height │
├─────────────────────────────────────────────────────────────────────────────────

# Inspect features

In [11]:
interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map=variable_names);

▣ (max(cntrd) < 1175.14) ∧ (min(mel8=1194Hz) < -4.25) ∧ (std(mel13=2832Hz) < 0.13) ∧ (std(mel9=1419Hz) < 0.28)  ↣  Pneumonia : (ninstances = 101, ncovered = 28, coverage = 0.28, confidence = 0.96, lift = 1.91, natoms = 4)
▣ (max(cntrd) ≥ 1175.14) ∧ (std(flatn) ≥ 0.06)  ↣  Healthy : (ninstances = 101, ncovered = 25, coverage = 0.25, confidence = 0.96, lift = 1.94, natoms = 2)


In [12]:
interesting_features = unique(SoleData.feature.(SoleLogics.value.(vcat(SoleLogics.atoms.(i.antecedent for i in interesting_rules)...))))
interesting_variables = sort(SoleData.i_variable.(interesting_features))

5-element Vector{Symbol}:
 Symbol("max(cntrd)")
 Symbol("min(mel8=1194Hz)")
 Symbol("std(flatn)")
 Symbol("std(mel13=2832Hz)")
 Symbol("std(mel9=1419Hz)")